In [341]:
import matplotlib.pyplot as plt
import csv
import os
import numpy as np
%matplotlib inline

In [342]:
class Sample:
    def __init__(self, label, processed_csv):
        #A sample contains the label name and the accelerometer (x,y,z) and gyroscope data (x,y,z) tuples
        #not necessary but its a bit more verbose to read than samples[label][0][0] for example
        self.label = label
        self.accelerometer = Sensor(processed_csv[0], processed_csv[1], processed_csv[2])
        self.gyroscope = Sensor(processed_csv[3], processed_csv[4], processed_csv[5])

class Sensor:
    def __init__(self, x, y, z):
        #again, not necessary but accessing it as accelerometer.x gives me the good brain chemicals
        self.x = x
        self.y = y
        self.z = z
    def print_range(self, num):
        #prints the first num values of the sensor
        print(self.x[:num])
        print(self.y[:num])
        print(self.z[:num])

In [343]:
def clean_csv(csv_file):
    #get labels
    #we won't really need them since we know the order of the data but it's a sanity check
    labels = csv_file[0]
    #swap rows and columns, excluding the first row
    #this makes it so csv_file[0] is the first column, csv_file[1] is the second column, etc.
    #this makes it way more intuitive to access the data
    csv_file = [list(i) for i in zip(*csv_file[1:])] 
    
    for i in range(6): #we only have 6 columns, but the standing code had extras past that so I'm hard coding it. Sorry.
        #convert strings to floats
        for j in range(len(csv_file[i])):
            try:
                csv_file[i][j] = float(csv_file[i][j])
            except:
                print("Error converting to float: " + str(csv_file[i][j]))

    return labels, csv_file


In [344]:
#we need to load the csvs but standup is contained in a folder since it would be harder to splice since standing happens at variable times
basedir = "data/"
labels = ["standup", "climbing_down", "climbing_up", "walking", "still"]
#we will break our data into 50 samples per label (since standup is not logically divisible or concatenable and it has 50 samples
#this means that the other samples, which are in a single file each, will be need to be roughly split into 50 samples each
data = [] #normally we seperate based on label but the Sample class stores that so we can deal with it later and just dump everything in here
for label in labels:
    if label == "standup":
        for filename in os.listdir(basedir+label):
            if filename.endswith(".csv"):
                with open(basedir+label+"/"+filename, 'r') as f:
                    reader = csv.reader(f)
                    csv_file = list(reader)
                    x, csv_file = clean_csv(csv_file)
                    new_sample = Sample(label, csv_file)
                    data.append(new_sample)
                    # print_csv(csv_file)
    else:
        with open(basedir+label+".csv", 'r') as f:
            reader = csv.reader(f)
            csv_file = list(reader)
            x, csv_file = clean_csv(csv_file)
            for i in range(50): #split into 50 samples
                #the exact length of the csv's will vary so we need to use split it into 50 and discard the remainder
                # print(i, "start:stop", i*(len(whole_csv_file)-1)//50, (i+1)*(len(whole_csv_file)-1)//50)
                index_range = range(i*(len(csv_file[0]))//50,(i+1)*(len(csv_file[0])//50))
                #because I swapped rows and columns, accessing the data is gross
                #this code is getting the window of data we want using that index range
                new_sample = Sample(label, [csv_file[0][index_range[0]:index_range[-1]], csv_file[1][index_range[0]:index_range[-1]], csv_file[2][index_range[0]:index_range[-1]], csv_file[3][index_range[0]:index_range[-1]], csv_file[4][index_range[0]:index_range[-1]], csv_file[5][index_range[0]:index_range[-1]]])
                data.append(new_sample)
                
                

#now we have all of our data in a list of Sample objects
print("data length:", len(data)) #should be 50*#labels - sanity check
           

data length: 250


In [345]:
#lets print some random label data
# for x in range (5):
#     sample = data[np.random.randint(0, len(data))]
#     print("label:", sample.label)
#     sample.accelerometer.print_range(5)
#     sample.gyroscope.print_range(5)

In [346]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *
from sklearn.preprocessing import Normalizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

In [347]:
#function structure taken from Assignment 1 - task 2
def featurize_input(data) -> np.array:
    out = []
    for sample in data:
        fv = []
        
        fv.append(np.min(sample.accelerometer.x))
        fv.append(np.min(sample.accelerometer.y))
        fv.append(np.min(sample.accelerometer.z))
        fv.append(np.max(sample.accelerometer.x))
        fv.append(np.max(sample.accelerometer.y))
        fv.append(np.max(sample.accelerometer.z))
        fv.append(np.ptp(sample.accelerometer.x))
        fv.append(np.ptp(sample.accelerometer.y))
        fv.append(np.ptp(sample.accelerometer.z))
        
        fv.append(np.mean(sample.gyroscope.x))
        fv.append(np.mean(sample.gyroscope.y))
        fv.append(np.mean(sample.gyroscope.z))
        fv.append(np.ptp(sample.gyroscope.x))
        fv.append(np.ptp(sample.gyroscope.y))
        fv.append(np.ptp(sample.gyroscope.z))

        out.append(fv)

    return np.array(out)


In [348]:
featurized_data = featurize_input(data)
print(featurized_data.shape)
labels = [sample.label for sample in data]

print(np.array(labels).shape)



(250, 15)
(250,)


In [359]:
# run this code

xtrain, xtest, ytrain, ytest = train_test_split(featurized_data, labels, test_size=0.30)

#training the model
clf = SVC() #note the default kernel here is 'rbf' - radial basis function
clf.fit(xtrain, ytrain)
cv_scores = cross_val_score(clf, xtrain, ytrain, cv=10)
print('Average Cross Validation Score from Training:', cv_scores.mean(), sep='\n', end='\n\n\n')

#testing the model
ypred = clf.predict(xtest)
cm = confusion_matrix(ytest, ypred)
cr = classification_report(ytest, ypred)

print('Confusion Matrix:', cm, sep='\n', end='\n\n\n')
print('Test Statistics:', cr, sep='\n', end='\n\n\n')

#This is what we will be grading (>95 expected)
print('Testing Accuracy:', accuracy_score(ytest, ypred))

Average Cross Validation Score from Training:
0.9084967320261439


Confusion Matrix:
[[12  2  0  0  0]
 [ 0 11  0  0  0]
 [ 0  0 19  0  0]
 [ 0  0  0 16  0]
 [ 0  0  0  0 15]]


Test Statistics:
               precision    recall  f1-score   support

climbing_down       1.00      0.86      0.92        14
  climbing_up       0.85      1.00      0.92        11
      standup       1.00      1.00      1.00        19
        still       1.00      1.00      1.00        16
      walking       1.00      1.00      1.00        15

     accuracy                           0.97        75
    macro avg       0.97      0.97      0.97        75
 weighted avg       0.98      0.97      0.97        75



Testing Accuracy: 0.9733333333333334
